# Machine Learning in Finance - Models Training

This notebook will focus on *how well we can forecast closing stocks using machine learning methods*.

*Authors:* [Mina Attia](https://people.epfl.ch/mina.attia), [Arnaud Felber](https://people.epfl.ch/arnaud.felber), ??? ??? & [Paulo Ribeiro](https://people.epfl.ch/paulo.ribeirodecarvalho)

## Import

Import all python scripts and/or libraries needed.

## Data

Tell where we use the data and describe it quickly. Better description will be given in the README.md file.

## Methods

Describe which method we will train and test.

### Random Model

### OLS

### Neural Network

### Transformers

## Results

We will test our models and compare them to see which one outperform the others.